In [100]:
from cassandra.cluster import Cluster
import re

In [160]:
class create_cassandra_instance:
    def __init__(self):
        cluster = Cluster([('localhost', 12001), ('localhost', 12002)])
        self.instance = cluster.connect()

    ########################################################################################################
    ########################################################################################################

    # FREELY EXECUTE ANY CQL QUERY
    def query(self, query):
        return self.instance.execute(query)

    ########################################################################################################
    ########################################################################################################
    
    # CASSANDRA DRIVER ERRORS ARE VERY MESSY
    # THIS ATTEMPT TO PARSE THEM TO MAKE EVERYTHING MORE HUMAN-READABLE
    def parse_error(self, error):
        stringified_error = str(error)
        
        # TRY TO REGEX MATCH THE ERROR PATTERN
        match = re.search(r'message="(.+)"', stringified_error)

        # MATCH FOUND, RETURN ISOLATED ERROR MSG
        if match:
            return match.group(1)
        
        # OTHERWISE, RETURN THE WHOLE THING
        return stringified_error
    
    ########################################################################################################
    ########################################################################################################

    # READ DATA FROM THE DATABASE
    def read(self, query: str) -> list[dict]:
        try:
            container = []

            # PERFORM THE TABLE QUERY
            query_result = self.instance.execute(query)

            # PARSE EACH ROW AS A DICT
            for item in query_result:
                container.append(item._asdict())

            return container
        
        # SAFELY CATCH ERRORS
        except Exception as raw_error:
            parsed_error = self.parse_error(raw_error)
            print(f'CASSANDRA READ ERROR: {parsed_error}')
    
    ########################################################################################################
    ########################################################################################################

    # FULL DATABASE OVERVIEW (KEYSPACES)
    def write(self, keyspace_table: str, row: dict):
        try:

            # SPLIT THE KEYS & VALUES
            columns = list(row.keys())
            values = list(row.values())

            # STITCH TOGETHER THE QUERY STRING
            query_string = f'INSERT INTO {keyspace_table} ('
            query_string += ', '.join(columns)
            query_string += ') values ('
            query_string += ', '.join(['?'] * len(columns))
            query_string += ');'
            
            # CONSTRUCT A PREPARED STATEMENT & EXECUTE THE DB WRITE
            prepared_statement = self.instance.prepare(query_string)
            self.instance.execute(prepared_statement, values)
        
        # SAFELY CATCH ERRORS
        except Exception as raw_error:
            parsed_error = self.parse_error(raw_error)
            print(f'CASSANDRA WRITE ERROR: {parsed_error}')

### CREATE A CASSANDRA INSTANCE

In [140]:
cassandra = create_cassandra_instance()

### READ DATA FROM DB

In [158]:
cassandra.read('SELECT * FROM testing_keyspace.testing_table')

[{'timestamp': 123126,
  'close': 1.100000023841858,
  'high': 3.299999952316284,
  'low': 4.400000095367432,
  'open': 2.200000047683716,
  'volume': 5000},
 {'timestamp': 123123,
  'close': 1.100000023841858,
  'high': 3.299999952316284,
  'low': 4.400000095367432,
  'open': 2.200000047683716,
  'volume': 5000},
 {'timestamp': 123127,
  'close': 1.100000023841858,
  'high': 3.299999952316284,
  'low': 4.400000095367432,
  'open': 2.200000047683716,
  'volume': 5000}]

### WRITE DATA TO DB

In [159]:
cassandra.write('testing_keyspace.testing_table', {
    'timestamp': 123127,
    'close': 1.1,
    'open': 2.2,
    'high': 3.3,
    'low': 4.4,
    'volume': 5000
})

In [161]:
import random

In [189]:
int(random.uniform(1, 1000000))

559678